# Finding Lane Lines

In this project we are going to implement the techniques using OpenCV library for identifying the lane lines on road , so that it is easy for car to follow the lanes and ride in between to avoid accidents.

## Steps to Achieve the Objective 

Following steps are performed to identify the lanes on images and later on application on video clips provdided for testing-
1. Grey Scale Conversion of image
2. Gussian Noise Removal
3. Edge Detection (Cannys Method)
4. Region of Interest
5. Line Detection (Houghs Transform)
6. Lane Formation 

### 1. Grey Scale Conversion

##### Reason for converting to grey scale -
1. Less computing power as now image is represented in less bytes as compared to color
2. Easy to track the changes in E.g. brightness, contrast, edges, shape, contours, texture, perspective, shadows etc. as we will be talking about only one color's shade

##### How do we make conversion - 
So, here comes the Opencv library with inbuilt functions to simplify the process. Function for same                             "cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)"


### 2. Gaussian Noise Removal

It is similar to convolution layer application in neural nets for image processing.

#### Why we need this -
1. To remove the noise in image which follows gaussian probability distribution (normal distribution), bell curve distribution
2. It removes the high frequency components of the image hence blurring the image a bit and removal of unwanted components 
3. It smoothens the image which will be helpful in edge detection (we will know it in next section)

#### Cons of this method -
1. It leads to reduction in details which might be important while detecting edges

#### How to do it -
Function is "cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)".
1. kernel size : Area that we want to take in consideration while detecting noisy components , larger its number less sensitive the filter will be

### 3. Edge Detection - Canny's Algorithm

#### Principle Behind the Algorithm -
It is based on calculating the gradient while traversing through the image , so when there is an edge/line there will be significant change in gradient of the pixel intensity, which helps in identifying the edges. It helps in identifying edges in vertical , horizontal and diagonal direction as well

#### How does it works - 
1. It calculates first order derivative(Gradient/slope) in horizonatal and vertical direction to identify the edge and it's orientation
2. It compares the gradient at one location with its neighbors and keep the maximum and masks others to identify the strogest edge . So this will keep only pixels with edges
3. Now to finalize on edges , it uses some minimum and maximum value cut off to conside it as an edge which will be user input. It is known as hyteresis thresholding

#### How to implement it -
Function is "cv2.Canny(img, low_threshold, high_threshold)"
1. Low threshold : intensity gradient below which edges will not be considered
2. High threshold: intensity gradient above which edges are sure


### 4. Region of Interest -

#### Why do we need it -
It is a method to mask the unwanted part of image so as to refine the further process of line detection as it will lead to reduction in noisy edges in surrounding of the road.

#### Mistake not to make -
I have thought of applying it at the start and then proceed for further steps , so as to reduce the computation.But, then i have realised that when we mask the surrounding area in some polygonal form it becomes strong edge hence line will be detected at those polygons edges . So I transferred it from start to after edge detection

#### How to apply -
We need to select the coordinated in image which needs to be masked and then apply the color filter over it to mask it 


### 5. Line Detection - Hough's Transform

#### What is Hough's Transform -
It is conversion from x-y coordinate system to new coordinate system, we can call it a,b coordinate system .It is just the transformed version of x-y. For example: lines equation in x-y system will be "y=mx+c" and we have defines new system as m-c system so the lines equation in new system becomes "c= y-mx".So, two inferences about conversion -
1. A point in x-y system will be a line in m-c system 
2. A line in x-y system will be a point in m-c system

With above two inferences question arises, what happens when slope is infinity that is line is parallel to y-axis in x-y system.For this Hough's transform can be represented in polar coordinates that is rho and thetha -
1. rho - perpendicular distance of the line from origin
2. thetha - Angle between perpendicular and x-axis

#### How it is used in line Detection -
So, the result from previous section gives us the edges , and we need to identify the lines in those edges .So it projects all the edge pixels in Hough's space , where if all pixels are on same line then they will intersect in Hough's space . Which leads to line detection for that combination of pixels .

#### How to do it -
Function "cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)"
1. rho - how much distance line from x-axis is acceptable
2. theta -  how much inclination from x-axis
3. threshold - min number of intersection on hough's space required to call it a line
4. minLineLength - min number of pixels to consider for calling it a line
5. maxLineGap - maximum gap between edge segements required to call it a line


### 6. Lane Formation (Not any famously known method)

After detecting lines in image we can see where lanes are but these lines are in segment , we need to connect them to create continuous lanes to define the boundary for car to ride

I have used used following approach based on differentiation in slopes of right and left lane :
1. According to images right lane has positive slope and left has negative slope 
2. Club the lines detected from previous section according to slope sign in right ans left group 
3. Now, as we can see from figure that both the lines has slope more then 0.5 , so we will discard the lines from clubbed ones which have slope less then 0.5
4. we have left and right groups of line segments.Now, how ti create a single line out of it . Initially i was trying to fit linear regression with one variable but then while searching i came across numpy function which is polynomial fit , in which we can define the degree of polynomial (in our case 1 as two axis line) and it creates the best line out of given points
6. From above we get linear equation for left and right lanes , now we need start and end coordinates for left and right lane.For this I have taken y of image shape as starting y coordinate and x can be calculated accordingly with the equations. For starting y coordinated , as presented in images it looked like it should be between 300 to 400 value where lane ends , hence i have tuned this to get the best possible result.
5. Finally, lanes are created , now to represent it in desired format i have increased the thickness of line in draw line function and coefficient for overlapping the images from 1 to less for transformed image to make it more transparent

Here you go solution for first assignment

#### Please Note : Some parameter tuning required for following methods :

1. Kernel Size for Gaussian Noise Removal
2. Low and high threshold for Canny's edge detection
3. threshold, minLineLength and maxLineGap for Hough's Transform 
4. Min value of Y in Lane formation to get the end point coordiantes of lane